## Importing used libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from preprocessing import preprocess_data, preprocess_data_relevant
from implementations import *
from utils import *

## Loading the data

First, we are going to load the data that we'll feed to our alorithm in order for it to learn. We have 3 .csv files to import: <br> *x_train.csv*, *x_test.csv* and *y_train.csv*.

In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("/Users/sarazatezalo/Documents/EPFL/semester 1/ML_course/projects/project1/data/dataset_to_release 2")

In [5]:
N_class1 = np.sum((y_train==1).astype(int))
N_class0 = np.sum((y_train==-1).astype(int))
N = len(y_train)
w0 = N/2/N_class1
w1 = N/2/N_class0
print('Optimal values of class weights are- for class 0:', w0, ', for class 1:', w1)

Optimal values of class weights are- for class 0: 5.662381363244176 , for class 1: 0.5484272630030753


### 1) When preprocessed with the whole set of features

In [6]:
# Predictions using logistic regression!

gammas = np.logspace(-5, -1, 5)
lambdas = np.logspace(-4, -1, 5)
w0s = [0.54, 0.7, 0.85, 0.9]
w1s = [5.56, 3.5, 1.5, 1]

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
print(x_train_pp.shape)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 25

best_lambda, best_gamma, best_acc, best_weights, best_f1, best_w0, best_w1 = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters, w0s, w1s)

print(f'Optimal values found are: best lambda: {best_lambda}, best gamma: {best_gamma}, best weight for class 0: {best_w0}, for class 1: {best_w1}')

print(f'Best accuracy on {k_fold}-fold: ', best_acc)
print(f'Best F1 score on {k_fold}-fold for class 1 ', best_f1)


(328135, 142)


/Users/sarazatezalo/Documents/EPFL/semester 1/ml-project-1-girl-mse/utils.py:32: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Optimal values found are: best lambda: 0.0001, best gamma: 0.01, best weight for class 0: 0.85, for class 1: 1.5
Best accuracy on 5-fold:  0.8585125024761151
Best F1 score on 5-fold for class 1  0.3699477262617819


In [7]:
lambdas = [0.001]
gammas = [0.01]
w0s = [0.55, 0.6, 0.7, 0.75, 0.85, 0.9, 0.95]
w1s = [1, 1.25, 1.5, 1.65, 1.7, 2, 3, 4]

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
print(x_train_pp.shape)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 25

for w0 in w0s:
    for w1 in w1s:
       _, _, best_acc, _, best_f1, _, _ = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters, [w0], [w1])
       print(f'For the w0:{w0}, and w1:{w1} we get F1 score of: {best_f1}, and accuracy of: {best_acc}')


(328135, 112)
For the w0:0.55, and w1:1 we get F1 score of: 0.36998281302259894, and accuracy of: 0.8563182836332608
For the w0:0.55, and w1:1.25 we get F1 score of: 0.36312478953531346, and accuracy of: 0.8722111326131012
For the w0:0.55, and w1:1.5 we get F1 score of: 0.35025556122105733, and accuracy of: 0.8831121337254484
For the w0:0.55, and w1:1.65 we get F1 score of: 0.340940801100984, and accuracy of: 0.8882228351135966
For the w0:0.55, and w1:1.7 we get F1 score of: 0.3377265072660231, and accuracy of: 0.8896521248876225
For the w0:0.55, and w1:2 we get F1 score of: 0.3133329428069812, and accuracy of: 0.896768098496046
For the w0:0.55, and w1:3 we get F1 score of: 0.21038721711963312, and accuracy of: 0.9091806725890258
For the w0:0.55, and w1:4 we get F1 score of: 0.10078628914685268, and accuracy of: 0.9118746857238635
For the w0:0.6, and w1:1 we get F1 score of: 0.37174301322338377, and accuracy of: 0.8513751961844973
For the w0:0.6, and w1:1.25 we get F1 score of: 0.36493

In [3]:
lambdas = [0.001]
gammas = [0.01]
w0s = [0.55, 0.75, 0.85, 0.95]
w1s = [1, 1.25, 1.5, 1.7, 2]

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
print(x_train_pp.shape)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 25

for w0 in w0s:
    for w1 in w1s:
       _, _, best_acc, _, best_f1, _, _ = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters, [w0], [w1])
       print(f'For the w0:{w0}, and w1:{w1} we get F1 score of: {best_f1}, and accuracy of: {best_acc}')

(328135, 111)
For the w0:0.55, and w1:1 we get F1 score of: 0.36993021210342264, and accuracy of: 0.8577079555670684
For the w0:0.55, and w1:1.25 we get F1 score of: 0.3620314232474455, and accuracy of: 0.8729364438417114
For the w0:0.55, and w1:1.5 we get F1 score of: 0.3490908067343088, and accuracy of: 0.883666783488503
For the w0:0.55, and w1:1.7 we get F1 score of: 0.3358292634188623, and accuracy of: 0.8901305864964115
For the w0:0.55, and w1:2 we get F1 score of: 0.3117181426118859, and accuracy of: 0.897121611531839
For the w0:0.75, and w1:1 we get F1 score of: 0.37321040762893826, and accuracy of: 0.8376369482072927
For the w0:0.75, and w1:1.25 we get F1 score of: 0.37076234202142916, and accuracy of: 0.8573879653191522
For the w0:0.75, and w1:1.5 we get F1 score of: 0.36487539987891565, and accuracy of: 0.8715802946957807
For the w0:0.75, and w1:1.7 we get F1 score of: 0.35554747106213536, and accuracy of: 0.8797994727779725
For the w0:0.75, and w1:2 we get F1 score of: 0.338

In [7]:
# Prediction using best parameters and ridge logistic regression
y_prediction = np.array([-1 if sigmoid(x.T @ best_weights) < 0.5 else 1 for x in x_test_pp])
print(y_prediction.shape)

(109379,)


In [8]:
print('There are this much -1s: ', np.sum((y_prediction==-1).astype(int)))
print('There are this much 1s: ', np.sum((y_prediction==1).astype(int)))


There are this much -1s:  93246
There are this much 1s:  16133


In [9]:
create_csv_submission(test_ids, y_prediction, "prediction_18.csv")

In [3]:
# For best parameters, different preprocessing - added the expansion with pairs for random 10 features

gammas = np.logspace(-5, -1, 5)
lambdas = np.logspace(-4, -1, 5)
w0s = [0.85]
w1s = [1.5]

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
print(x_train_pp.shape)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 50

best_lambda, best_gamma, best_acc, best_weights, best_f1, best_w0, best_w1 = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters, w0s, w1s)

print(f'Best accuracy on {k_fold}-fold: ', best_acc)
print(f'Best F1 score on {k_fold}-fold for class 1 ', best_f1)

(328135, 156)
Best accuracy on 5-fold:  0.8422996632483581
Best F1 score on 5-fold for class 1  0.3461833326817825


In [4]:
# Prediction using ridge linear regression

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=-1)
lambdas = np.logspace(-4, 0, 10)
k_fold = 5
best_lambda, best_acc, best_weights, best_f1 = best_cv_ridge_reg(y_train_pp, x_train_pp, k_fold, lambdas)
y_train_prediction = np.array([-1 if x.T @ best_weights < 0 else 1 for x in x_train_pp])

print('Accuracy on KFold for the best model: ', best_acc)
print('F1 on KFold for the best model: ', best_f1)


Accuracy on KFold for the best model:  0.9052188885672056
F1 on KFold for the best model:  0.2177996772608644


In [3]:
# Prediction using ridge linear regression with expansion with pairs of random 10 featues

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=-1)
lambdas = np.logspace(-4, 0, 10)
k_fold = 5
best_lambda, best_acc, best_weights, best_f1 = best_cv_ridge_reg(y_train_pp, x_train_pp, k_fold, lambdas)
y_train_prediction = np.array([-1 if x.T @ best_weights < 0 else 1 for x in x_train_pp])

print('Accuracy on KFold for the best model: ', best_acc)
print('F1 on KFold for the best model: ', best_f1)


Accuracy on KFold for the best model:  0.9071814954210918
F1 on KFold for the best model:  0.27084897363875127


### 2) When preprocessed with a set of relevant features:

In [3]:
# Predictions using logistic regression!

gammas = np.logspace(-5, -1, 3)
lambdas = np.logspace(-4, -1, 3)
w0s = [0.54, 0.7, 0.85, 0.9]
w1s = [5.56, 3.5, 1.5, 1]

x_train_pp, x_test_pp, y_train_pp = preprocess_data_relevant(x_train, y_train, x_test, neg_label=0)
print(x_train_pp.shape)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 25

best_lambda, best_gamma, best_acc, best_weights, best_f1, best_w0, best_w1 = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters, w0s, w1s)

print(f'Optimal values found are: best lambda: {best_lambda}, best gamma: {best_gamma}, best weight for class 0: {best_w0}, for class 1: {best_w1}')

print(f'Best accuracy on {k_fold}-fold: ', best_acc)
print(f'Best F1 score on {k_fold}-fold for class 1 ', best_f1)


(328135, 109)


/Users/sarazatezalo/Documents/EPFL/semester 1/ml-project-1-girl-mse/implementations.py:186: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-t))
/Users/sarazatezalo/Documents/EPFL/semester 1/ml-project-1-girl-mse/implementations.py:208: RuntimeWarning: divide by zero encountered in log
  5.66 * y * np.log(sigmoid(tx.dot(w))) + 0.54 * (1 - y) * (np.log(1 - sigmoid(tx.dot(w))))
/Users/sarazatezalo/Documents/EPFL/semester 1/ml-project-1-girl-mse/implementations.py:208: RuntimeWarning: invalid value encountered in multiply
  5.66 * y * np.log(sigmoid(tx.dot(w))) + 0.54 * (1 - y) * (np.log(1 - sigmoid(tx.dot(w))))
/Users/sarazatezalo/Documents/EPFL/semester 1/ml-project-1-girl-mse/utils.py:34: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * (precision * recall)/(precision+recall)


Optimal values found are: best lambda: 0.0031622776601683794, best gamma: 0.1, best weight for class 0: 0.85, for class 1: 1.5
Best accuracy on 5-fold:  0.9035549392780411
Best F1 score on 5-fold for class 1  0.1033976046939479


In [4]:
# Predictions using ridge linear regression
x_train_pp, x_test_pp, y_train_pp = preprocess_data_relevant(x_train, y_train, x_test, neg_label=-1)

lambdas = np.logspace(-4, 0, 10)
k_fold = 5
best_lambda, best_acc, best_weights, best_f1 = best_cv_ridge_reg(y_train_pp, x_train_pp, k_fold, lambdas)
y_train_prediction = np.array([-1 if x.T @ best_weights < 0 else 1 for x in x_train_pp])

print('Accuracy on KFold for the best model: ', best_acc)
print('F1 on KFold for the best model: ', best_f1)

Accuracy on KFold for the best model:  0.912813323784418
F1 on KFold for the best model:  0.05572685097485919
